# Trabalho de Conclusão de Curso


## Criação e tratamento da base de dados

### Importando Bibliotecas

In [ ]:
import tkinter as tk
from tkinter import ttk
import webbrowser
import pandas as pd
import tkinter.messagebox as tkm
import threading
import time

### Definindo dicionário da base do SISU

In [ ]:
bases_sisu = {
    "2016-1": {"ano": 2016, "semestre": 1, "encoding": "utf-8", "delimiter": "|"},
    "2016-2": {"ano": 2016, "semestre": 2, "encoding": "utf-8", "delimiter": "|"},
    "2017-1": {"ano": 2017, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2017-2": {"ano": 2017, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2018-1": {"ano": 2018, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2018-2": {"ano": 2018, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2019-1": {"ano": 2019, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2019-2": {"ano": 2019, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2020-1": {"ano": 2020, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2020-2": {"ano": 2020, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2021-1": {"ano": 2021, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2021-2": {"ano": 2021, "semestre": 2, "encoding": "cp1252", "delimiter": "|"},
    "2022-1": {"ano": 2022, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2022-2": {"ano": 2022, "semestre": 2, "encoding": "cp1252", "delimiter": "|"},
}

### Selecionando colunas de interesse

In [3]:
def selecionar_colunas(chave, dados_sisu, dados_enem):

    # Selecionar Colunas ENEM e criar uma cópia
    enem = dados_enem[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'TP_SEXO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q022', 'Q024', 'Q025']].copy()

    # Renomear Colunas ENEM
    enem = enem.rename(columns={'NU_NOTA_CN': 'CN', 'NU_NOTA_CH': 'CH', 'NU_NOTA_LC': 'LC', 'NU_NOTA_MT': 'MT', 'NU_NOTA_REDACAO': 'REDACAO', 'TP_SEXO': 'SEXO'})

    # Selecionar e renomear colunas SISU
    if chave in ["2016-1", "2016-2"]:
        sisu = dados_sisu[['nu_nota_cn', 'nu_nota_ch', 'nu_nota_l', 'nu_nota_m', 'nu_nota_r', 'nota_concorrencia', 'nu_notacorte', 'tp_sexo', 'nome_curso', 'dt_nascimento', 'uf_origem', 'municipio_candidato', 'st_aprovado', 'nu_ano', 'nu_edicao', 'sigla_ies', 'campus']].copy()
        sisu = sisu.rename(columns={'nu_nota_l': 'LC', 'nu_nota_ch': 'CH', 'nu_nota_cn': 'CN', 'nu_nota_m': 'MT', 'nu_nota_r': 'REDACAO'})
        
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = dados_sisu[['NOTA_CN', 'NOTA_CH', 'NOTA_L', 'NOTA_M', 'NOTA_R', 'NOTA_CANDIDATO', 'NOTA_CORTE', 'SEXO', 'NOME_CURSO', 'DATA_NASCIMENTO', 'UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'APROVADO','ANO', 'EDICAO', 'SIGLA_IES', 'NOME_CAMPUS']].copy()
        sisu = sisu.rename(columns={'NOTA_L': 'LC', 'NOTA_CH': 'CH', 'NOTA_CN': 'CN', 'NOTA_M': 'MT', 'NOTA_R': 'REDACAO'})

    else:
        sisu = dados_sisu[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_L', 'NU_NOTA_M', 'NU_NOTA_R', 'NU_NOTA_CANDIDATO', 'NU_NOTACORTE', 'TP_SEXO', 'NO_CURSO', 'DT_NASCIMENTO', 'SG_UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'ST_APROVADO', 'NU_ANO', 'NU_EDICAO', 'SIGLA_IES', 'NO_CAMPUS']].copy()
        sisu = sisu.rename(columns={'NU_NOTA_L': 'LC', 'NU_NOTA_CH': 'CH', 'NU_NOTA_CN': 'CN', 'NU_NOTA_M': 'MT', 'NU_NOTA_R': 'REDACAO'})

    return sisu, enem

### Filtrando a base por instituição

In [ ]:
def filtrar_instituicao(chave, sisu, cod_instituicao):
    if chave in ["2016-1", "2016-2"]:
        sisu = sisu[sisu["cod_ies"] == cod_instituicao]
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = sisu[sisu['CODIGO_IES'] == cod_instituicao]
    else:
        sisu = sisu[sisu['CO_IES'] == cod_instituicao]
    
    return sisu

### Filtrando a base por campus

In [ ]:
def filtrar_campus(chave, sisu, cod_campus):
    if chave in ["2016-1", "2016-2"]:
        sisu = sisu[sisu["cod_campus"] == cod_campus]
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = sisu[sisu['CODIGO_CAMPUS'] == cod_campus]
    else:
        sisu = sisu[sisu['CO_CAMPUS'] == cod_campus]
    
    return sisu

### Processando base (Subsituição de caracteres, preenchimento de linhas em branco, conversão de tipos de dados)

In [ ]:
def processar_base(chave, sisu, enem):
    
    colunas = ['CH', 'CN', 'MT', 'LC', 'REDACAO']
    
    for col in colunas:
        if col in sisu.columns:
            sisu.loc[:, col] = sisu[col].astype(str).str.replace(',', '.').astype(float)
    
    colunas_com_nan = sisu.columns[sisu.isnull().any()].tolist()

    if len(colunas_com_nan) > 0:
        if chave in ["2016-1", "2016-2"]:
            sisu['nu_notacorte'] = pd.to_numeric(sisu['nu_notacorte'], errors='coerce')
            sisu['nu_notacorte'] = sisu.groupby('nome_curso')['nu_notacorte'].transform(lambda x: x.fillna(x.mean()))

        elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
            sisu['NOTA_CORTE'] = pd.to_numeric(sisu['NOTA_CORTE'], errors='coerce')
            sisu['NOTA_CORTE'] = sisu.groupby('NOME_CURSO')['NOTA_CORTE'].transform(lambda x: x.fillna(x.mean()))

        else:
            sisu['NU_NOTACORTE'] = pd.to_numeric(sisu['NU_NOTACORTE'], errors='coerce')
            sisu['NU_NOTACORTE'] = sisu.groupby('NO_CURSO')['NU_NOTACORTE'].transform(lambda x: x.fillna(x.mean()))


    for col in colunas:
        if col in enem.columns:
            enem.loc[:, col] = enem[col].round(2).astype(float)
        if col in sisu.columns:
            sisu.loc[:, col] = sisu[col].round(2).astype(float)

    return sisu, enem

### Mesclar a base

In [ ]:
def mesclar_base(sisu, enem):
    return pd.merge(sisu, enem, on=['CH', 'CN', 'MT', 'LC', 'REDACAO'], how='inner')

## Funções e tratamentos relacionados a interface

### Link do eMEC

In [8]:
# Função para abrir o link no navegador
def abrir_link():
    webbrowser.open("https://emec.mec.gov.br/emec/nova")

### Tratamento dos parâmetros recebidos pela interface gráfica

In [ ]:
def parametros(ano_inicial, ano_final, semestre_inicial, semestre_final, cod_campus, cod_instituicao):
    # Obter os parâmetros
    ano_inicial = int(ano_inicial)
    semestre_inicial = int(semestre_inicial)
    ano_final = int(ano_final)
    semestre_final = int(semestre_final)

    if cod_instituicao != "":
        cod_instituicao = int(cod_instituicao)
    else:
        cod_instituicao = None

    if cod_campus != "":
        cod_campus = int(cod_campus)
    else:
        cod_campus = None

    return ano_inicial, ano_final, semestre_inicial, semestre_final, cod_campus, cod_instituicao

### Gerar vetor de edições que serão filtradas

In [ ]:
def gerar_edicoes(ano_inicial, semestre_inicial, ano_final, semestre_final):
    edicoes = []
    for ano in range(ano_inicial, ano_final + 1):
        for semestre in range(1, 3):  # Semestres 1 e 2
            if ano == ano_inicial and semestre < semestre_inicial:
                continue  # Pula semestres antes do inicial
            if ano == ano_final and semestre > semestre_final:
                continue  # Pula semestres após o final
            edicoes.append((ano, semestre))
    return edicoes

## Execução do Programa

### Execução da interface, cronômetro e criação da base de dados

In [ ]:
# Variáveis globais para o cronômetro
cronometro_ativo = False
tempo_inicial = 0

def atualizar_cronometro():
    if cronometro_ativo:
        tempo_decorrido = time.time() - tempo_inicial
        minutos = int(tempo_decorrido // 60)
        segundos = int(tempo_decorrido % 60)
        cronometro_label.config(text=f"Tempo decorrido: {minutos} minutos e {segundos} segundos")
        
        # Reexecuta esta função após 1000 ms (1 segundo)
        cronometro_label.after(1000, atualizar_cronometro)

def fechar_programa():
    root.destroy()

def gerar_base():
    global tempo_inicial, cronometro_ativo
    cronometro_ativo = True
    tempo_inicial = time.time()

    ano_inicial = var_ano_inicial.get()
    ano_final = var_ano_final.get()
    semestre_inicial = var_semestre_inicial.get()
    semestre_final = var_semestre_final.get()
    cod_instituicao = entry_cod_instituicao.get()
    cod_campus = entry_cod_campus.get()

    ano_inicial, ano_final, semestre_inicial, semestre_final, cod_campus, cod_instituicao = parametros(ano_inicial, ano_final, semestre_inicial, semestre_final, cod_campus, cod_instituicao)

    atualizar_cronometro()
    
    # Lista para armazenar as bases unidas
    todas_bases = []

    # Definindo as edições
    edicoes = gerar_edicoes(ano_inicial, semestre_inicial, ano_final, semestre_final)

    for ano, semestre in edicoes:
        chave = f"{ano}-{semestre}"

        # Definindo caminhos
        endereco_ENEM = f'./DADOS/ENEM/microdados_enem_{ano-1}/DADOS/microdados_enem_{ano-1}.csv'
        endereco_SISU = f'./DADOS/SISU/ListagemChamadaRegular_{ano}-{semestre}/ListagemChamadaRegular_{ano}-{semestre}.csv'
        delimiter_SISU = bases_sisu[chave]["delimiter"]
        encoding_SISU = bases_sisu[chave]["encoding"]

        # Carregar as bases
        dados_enem = pd.read_csv(endereco_ENEM, delimiter=";", encoding="cp1252")
        dados_sisu = pd.read_csv(endereco_SISU, delimiter=delimiter_SISU, encoding=encoding_SISU, low_memory=False)

        # Filtrando a base do SISU por instituicao
        if cod_instituicao != None:
            dados_sisu = filtrar_instituicao(chave, dados_sisu, cod_instituicao)

        # Filtrando a base do SISU por campus
        if cod_campus != None:
            dados_sisu = filtrar_campus(chave, dados_sisu, cod_campus)

        # Selecionar colunas de interesse
        sisu, enem = selecionar_colunas(chave, dados_sisu, dados_enem)

        # Processar as bases
        sisu, enem = processar_base(chave, sisu, enem)

        # Mesclar as bases
        base_mesclada = mesclar_base(sisu, enem)

        print(f'Base {ano}-{semestre} concluída!')
        print(f' - Nesta edição a UTFPR-CP obteve {len(sisu)} inscrições e a base mesclada tem {len(base_mesclada)} linhas')

        # Padronizando os nomes das colunas
        colunas_padronizadas = ['CN', 'CH', 'LC', 'MT', 'REDACAO', 'NOTA_GERAL', 'NOTA_CORTE', 'SEXO_S', 'CURSO', 'DATA DE NASCIMENTO', 'UF', 'MUNICIPIO', 'APROVADO','ANO', 'EDICAO', 'SIGLA_IES', 'CAMPUS','SEXO_E', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q022', 'Q024', 'Q025']
        base_mesclada.columns = colunas_padronizadas
        todas_bases.append(base_mesclada)

        # Atualizar a barra de progresso
        progress['value'] += 100 / len(edicoes)  # Atualizar progresso
        root.update_idletasks()  # Atualiza a interface


    # Concatenar todas as bases unidas
    Base_Final = pd.concat(todas_bases, ignore_index=True)
    Base_Final.insert(0, 'ID', range(1, len(Base_Final) + 1))

    # Exportar a base final para Excel
    Base_Final.to_excel('./BaseFinal.xlsx', index=False)

    # Mensagem de conclusão
    tkm.showinfo("Concluído", "A base de dados foi gerada com sucesso!")

    # Para parar o cronômetro
    cronometro_ativo = False
    cronometro_label.config(text="Processo concluído!")

# Configuração da janela principal
root = tk.Tk()
root.title("Gerador de Base de Dados Filtrada")
root.geometry("500x500")  # Aumentar o tamanho da janela
root.resizable(False, False)  # Não permitir maximização

# Opções de ano e semestre
anos = ["2016", "2017", "2018", "2019", "2020", "2021", "2022"]
semestres = ["1", "2"]

# Adicionando um frame para centralizar o conteúdo
frame = tk.Frame(root)
frame.pack(expand=True)

# Coleta de Ano Inicial
tk.Label(frame, text="Ano Inicial:").grid(row=0, column=0, padx=10, pady=5)
var_ano_inicial = tk.StringVar(value=anos[0])
lista_ano_inicial = tk.OptionMenu(frame, var_ano_inicial, *anos)
lista_ano_inicial.grid(row=0, column=1, padx=10, pady=5)

# Coleta de Ano Final
tk.Label(frame, text="Ano Final:").grid(row=1, column=0, padx=10, pady=5)
var_ano_final = tk.StringVar(value=anos[0])
lista_ano_final = tk.OptionMenu(frame, var_ano_final, *anos)
lista_ano_final.grid(row=1, column=1, padx=10, pady=5)

# Coleta de Semestre Inicial
tk.Label(frame, text="Semestre Inicial:").grid(row=2, column=0, padx=10, pady=5)
var_semestre_inicial = tk.StringVar(value=semestres[0])
lista_semestre_inicial = tk.OptionMenu(frame, var_semestre_inicial, *semestres)
lista_semestre_inicial.grid(row=2, column=1, padx=10, pady=5)

# Coleta de Semestre Final
tk.Label(frame, text="Semestre Final:").grid(row=3, column=0, padx=10, pady=5)
var_semestre_final = tk.StringVar(value=semestres[0])
lista_semestre_final = tk.OptionMenu(frame, var_semestre_final, *semestres)
lista_semestre_final.grid(row=3, column=1, padx=10, pady=5)

# Campo para o Código da Instituição
tk.Label(frame, text="Código da Instituição:").grid(row=4, column=0, padx=10, pady=5)
entry_cod_instituicao = tk.Entry(frame)
entry_cod_instituicao.grid(row=4, column=1, padx=10, pady=5)

# Campo para o Código do Campus
tk.Label(frame, text="Código do Campus:").grid(row=5, column=0, padx=10, pady=5)
entry_cod_campus = tk.Entry(frame)
entry_cod_campus.grid(row=5, column=1, padx=10, pady=5)

# Botão para gerar a base de dados
btn_gerar = tk.Button(frame, text="Gerar Base", command=lambda: threading.Thread(target=gerar_base).start())
btn_gerar.grid(row=6, column=0, columnspan=2, pady=10)

# Label com link clicável
label_link = tk.Label(frame, text="Não sabe o código do campus ou da instituição? Acesse AQUI", fg="blue", cursor="hand2")
label_link.grid(row=7, column=0, columnspan=2, pady=10)
label_link.bind("<Button-1>", lambda e: abrir_link())

# Label com aviso
aviso = tk.Label(frame, text="Tempo médio de espera: 3 minutos para cada semestre selecionado.")
aviso.grid(row=8, column=0, columnspan=2, pady=10)

# Label para o cronômetro
cronometro_label = tk.Label(frame, text="Tempo decorrido: 0.0 segundos")
cronometro_label.grid(row=9, column=0, columnspan=2, pady=10)

# Configuração da barra de progresso
progress = ttk.Progressbar(frame, orient="horizontal", length=300, mode='determinate')
progress.grid(row=10, column=0, columnspan=2, pady=20)

# Botão para fechar o programa
btn_fechar = tk.Button(frame, text="Fechar", command=fechar_programa)
btn_fechar.grid(row=11, column=0, columnspan=2, pady=10)

# Expandir o frame
frame.pack(expand=True)

root.mainloop()

Base 2016-1 concluída!
 - Nesta edição a UTFPR-CP obteve 3465 inscrições e a base mesclada tem 3465 linhas


Exception in thread Thread-47 (gerar_base):
Traceback (most recent call last):
  File "c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\gabri\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gabri\AppData\Local\Temp\ipykernel_7584\1103312626.py", line 79, in gerar_base
  File "c:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1728, in cget
    return self.tk.call(self._w, 'cget', '-' + key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop
